# Leitor de Planilhas da Vivo
https://github.com/jespimentel

OBJETIVO

1. A Vivo fornece em planilha Excel, mediante requisição judicial, os metadados das chamadas realizadas a partir de telefones celulares.

2. Na planilha da operadora é possível identificar as ERBs utilizadas nas chamadas pelas colunas comuns de duas diferentes pastas de trabalho.

3. Este script auxilia o investigador na tarefa de fazer esse relacionamento. Além disso, converte as coordenadas geográficas em formato legível ao My Maps do Google, ranqueia os interlocutores segundo o número de contatos feitos com o alvo (gerando gráficos de pizza) e plota em mapa a localização das ERBs usadas nas chamadas, tanto as do alvo, quanto as dos interlocutores, distinguindo-as. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

chamadas = pd.read_excel ('consulta_vivo.xlsx', sheet_name=0) # Altere esta linha para que corresponda ao caminho da sua planilha
informacoes_gerais = chamadas.iloc[2,1]
chamadas.columns = chamadas.iloc[4]
chamadas = chamadas[5:]
erbs = pd.read_excel ('consulta_vivo.xlsx', header=4, sheet_name=2) # Altere esta linha para que corresponda ao caminho da sua planilha
erbs.columns = erbs.iloc[0]
erbs = erbs[1:]
linha = informacoes_gerais[6:17]

In [ ]:
# Função para a conversão das coordenadas geográficas da planilha para o formato DD,
# visando à plotagem das localizações no Folium ou no Google Maps

def converte_dms_dd(coordenada):
  if coordenada[0] == '-':
    new_coord = coordenada[0]
  else:
    new_coord = ''
  coordenada = coordenada.replace(',', '.').split('-') # Ex. de resultado: ['', '22', '47', '35.5']
  if coordenada[0] == '':
    del (coordenada[0])
  graus = int(coordenada[0])
  decimais = float(coordenada[1])/60 + float(coordenada[2])/3600
  new_coord = new_coord + str(graus + decimais)
  return float(new_coord)

# Substituição das coordenadas convertidas em DD no Dataframe
erbs.Latitude = erbs.Latitude.map(lambda x: converte_dms_dd(x))
erbs.Longitude = erbs.Longitude.map(lambda x: converte_dms_dd(x))

In [ ]:
informacoes_gerais

In [ ]:
chamadas ['Data-hora'] = (chamadas['Data']) + '-' + (chamadas ['Hora'])
chamadas.head()

In [ ]:
erbs.head()

In [ ]:
# Chamadas realizadas pelo alvo
parametro = chamadas['Chamador'] == linha
numeros_chamados_pelo_alvo = chamadas[parametro]
numeros_chamados_pelo_alvo.head()

In [ ]:
# Chamadas recebidas pelo alvo
parametro = chamadas['Chamador'] != linha
numeros_recebidas_pelo_alvo = chamadas[parametro]
numeros_recebidas_pelo_alvo.head()

In [ ]:
# Localização do alvo nas chamadas por ele realizadas
merge_localizacao_alvo_como_chamador = pd.merge (numeros_chamados_pelo_alvo, erbs, left_on='Local Origem', right_on='CGI')
merge_localizacao_alvo_como_chamador[['Data', 'Hora', 'Chamado', 'CGI', 'UF', 'Cidade', 'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude', 'Data-hora']]

In [ ]:
# Localização do alvo nas chamadas por ele recebidas
merge_localizacao_alvo_recebidas = pd.merge (numeros_recebidas_pelo_alvo, erbs, left_on='Local Destino', right_on='CGI')
merge_localizacao_alvo_recebidas[['Data', 'Hora', 'Chamador', 'CGI', 'UF', 'Cidade', 'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude','Data-hora']]

In [ ]:
# Localização do alvo nas chamadas por ele realizadas e recebidas
# Geração do arquivo csv para plotagem
localizacao_alvo_chamadas_receb_e_realiz = pd.concat([merge_localizacao_alvo_recebidas, merge_localizacao_alvo_como_chamador])
localizacao_alvo_chamadas_receb_e_realiz = localizacao_alvo_chamadas_receb_e_realiz [['Data', 'Hora', 'Chamador', 'Chamado', 'CGI', 'UF', 'Cidade', 
                                                                                      'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude','Data-hora']]
localizacao_alvo_chamadas_receb_e_realiz = localizacao_alvo_chamadas_receb_e_realiz.set_index(['Data']).sort_values(['Data','Hora'])
localizacao_alvo_chamadas_receb_e_realiz.to_csv('coord_alvo.csv') # Altere o destino para adequá-lo à sua necessidade.
localizacao_alvo_chamadas_receb_e_realiz

In [ ]:
# Localização dos interlocutores nas chamadas feitas pelo alvo
merge_localizacao_interlocutor_alvo_como_chamador = pd.merge (numeros_chamados_pelo_alvo, erbs, left_on='Local Destino', right_on='CGI')
merge_localizacao_interlocutor_alvo_como_chamador[['Data', 'Hora', 'Chamado', 'CGI', 'UF', 'Cidade', 'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude','Data-hora']]

In [ ]:
# Localização dos interlocutores nas chamadas feitas por eles
merge_localizacao_interlocutor_como_chamador = pd.merge (numeros_recebidas_pelo_alvo, erbs, left_on='Local Origem', right_on='CGI')
merge_localizacao_interlocutor_como_chamador[['Data', 'Hora', 'Chamador', 'CGI', 'UF', 'Cidade', 'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude','Data-hora']]

In [ ]:
# Localização dos interlocutores em todas as chamadas
# Geração do arquivo csv para plotagem
localizacao_interlocutores_todas = pd.concat([merge_localizacao_interlocutor_alvo_como_chamador,merge_localizacao_interlocutor_como_chamador])
localizacao_interlocutores_todas = localizacao_interlocutores_todas[['Data', 'Hora', 'Chamador', 'Chamado', 'CGI', 'UF', 'Cidade', 
                                                                    'Bairro', 'Endereço', 'Azi', 'Latitude', 'Longitude','Data-hora']]
localizacao_interlocutores_todas = localizacao_interlocutores_todas.set_index(['Data']).sort_values(['Data','Hora'])
#localizacao_interlocutores_todas = localizacao_interlocutores_todas.sort_values(['Data','Hora'])
localizacao_interlocutores_todas.to_csv('coord_interlocutores.csv') # Altere o destino para adequá-lo à sua necessidade.
localizacao_interlocutores_todas

In [ ]:
# Ranking dos números chamados
#numeros_chamados = chamadas.groupby('Chamado')['Data'].count().sort_values(ascending=False)[1:]
numeros_chamados = chamadas.groupby('Chamado')['Chamado'].count().sort_values(ascending=False)[1:]
numeros_chamados

In [ ]:
plt.style.use('fivethirtyeight')
plt.title ('Ranking dos números chamados pelo alvo')
numeros_chamados.plot.pie(shadow = True, figsize = (10, 10), autopct='%1.1f%%')
plt.savefig('chamados.png', dpi = 300, bbox_inches='tight') # Altere o destino para adequá-lo à sua necessidade.
plt.show()

In [ ]:
# Ranking dos chamadores
chamadas_realizadas = chamadas.groupby('Chamador')['Chamador'].count().sort_values(ascending=False)[1:]
chamadas_realizadas

In [ ]:
plt.style.use('fivethirtyeight')
plt.title ('Ranking dos chamadores')
chamadas_realizadas.plot.pie(shadow = True, figsize = (10, 10), autopct='%1.1f%%')
plt.savefig('chamadores.png', dpi = 300, bbox_inches='tight') # Altere o destino para adequá-lo à sua necessidade.
plt.show()

In [ ]:
# Ranking dos números mais contatados (independentemente de quem fez a ligação)
ranking_total = numeros_chamados.add(chamadas_realizadas, fill_value=0).astype(int)
ranking_total.sort_values(ascending=False, inplace=True)
ranking_total

In [ ]:
plt.style.use('fivethirtyeight')
plt.title ('Ranking dos números mais contatados')
ranking_total.plot.pie(shadow = True, figsize = (10, 10), autopct='%1.1f%%')
plt.savefig('numeros_mais_contatados.png', dpi = 300, bbox_inches='tight') # Altere o destino para adequá-lo à sua necessidade.
plt.show()

In [ ]:
# Geração do relatório 'leitura_vivo.txt'
# Funcionalidade substituída pela planilha auxiliar (abaixo)

#relatorio = 'Leitura da Planilha Vivo\n' + informacoes_gerais + '\n\n'
#relatorio += 'Chamadas realizadas pelo alvo\n' + str(numeros_chamados_pelo_alvo[['Data', 'Hora', 'Chamado', 'Durac']].set_index('Data'))+ '\n\n'
#relatorio += 'Chamadas recebidas pelo alvo\n' + str(numeros_recebidas_pelo_alvo[['Data', 'Hora', 'Chamador', 'Durac']].set_index('Data'))+ '\n\n'
#relatorio += 'Localização das EBBs do alvo nas chamadas realizadas e recebidas\n' + str(localizacao_alvo_chamadas_receb_e_realiz[['Hora', 'Cidade', 'Endereço']]) + '\n\n'
#relatorio += 'Observação: o programa gerou o arquivo \"coord_alvo.csv\"  para plotagem no My Maps (do Google)\n\n'
#relatorio += 'Localização das EBBs dos interlocutores\n' + str(localizacao_interlocutores_todas[['Hora', 'Cidade', 'Endereço']]) + '\n\n'
#relatorio += 'Observação: o programa gerou o arquivo \"coord_interlocutores.csv\"  para plotagem no My Maps (do Google)\n\n'
#relatorio += 'Ranking das ligações totais\n' + str(ranking_total) + '\n\n'
#relatorio += 'Ranking dos números chamados pelos alvo\n' + str(numeros_chamados) + '\n\n'
#relatorio += 'Ranking dos chamadores\n' + str(chamadas_realizadas) + '\n\n'
#arquivo = open('leitura_vivo.txt','w') # Altere o destino para adequá-lo à sua necessidade.
#arquivo.write (relatorio + '\n')
#arquivo.close()

In [ ]:
# Criação de uma planilha Excel para resumir o trabalho
with pd.ExcelWriter('auxilia_leitura.xlsx') as writer:  
  numeros_chamados_pelo_alvo.to_excel(writer, sheet_name='Números chamados pelo alvo')
  numeros_recebidas_pelo_alvo.to_excel(writer, sheet_name='Números recebidos pelo alvo')
  localizacao_alvo_chamadas_receb_e_realiz.to_excel(writer, sheet_name='Localização do alvo')
  localizacao_interlocutores_todas.to_excel(writer, sheet_name='Localização dos interlocutores')
  numeros_chamados.to_excel(writer, sheet_name='Números mais chamados pelo alvo')
  chamadas_realizadas.to_excel(writer, sheet_name='Chamadores')
  ranking_total.to_excel(writer, sheet_name='Total')


In [ ]:
alvo = localizacao_alvo_chamadas_receb_e_realiz
alvo_lat = alvo['Latitude']
alvo_long = alvo['Longitude']
alvo_popup = alvo['Data-hora']

inter = localizacao_interlocutores_todas
inter_lat = inter['Latitude']
inter_long = inter['Longitude']
inter_popup = inter['Data-hora']

In [23]:
import folium

mapa = folium.Map(width=800, height=600,location=[-15.77972, -47.92972], zoom_start=4)
#mapa = folium.Map(location=[-15.77972, -47.92972], zoom_start=4)
for lat, lon, nom in zip(alvo_lat, alvo_long, alvo_popup):
    folium.Marker(location=[float(lat), float(lon)],                
                  popup=nom,icon=folium.Icon(color='red')).add_to(mapa)

for lat, lon, nom in zip(inter_lat, inter_long, inter_popup):
    folium.Marker(location=[float(lat), float(lon)],                
                  popup=nom,icon=folium.Icon(color='green')).add_to(mapa)                 
mapa.save('mapa.html') #Altere o local de acordo com a sua preferência
mapa